In [1]:
# Load the Julia environment needed for this notebook
import Pkg
Pkg.activate("../Tasks2D")

  Activating project at `~/projects/tasks2D/Tasks2D`


In [151]:
import GLMakie   # Visualization Library
using Dates

using Revise      # For development; makes it so modifications
# to imported modules are immediately reflected in this Julia session
using Gen         # Gen probabilistic programming library
import GenParticleFilters # Additional particle filtering functionality for Gen
import GridWorlds # Simple gridworld functionality
import LineWorlds as L # For ray casting
import GenPOMDPs  # Beginnings of a Gen POMDP library

includet("SLAM/Utils.jl")
includet("SLAM/distributions.jl")
GLMakie.activate!()

## Using Gen to define a probabilistic model of an agent and the map in a gridworld

First, we define a prior over the agent's initial position and possible map.

In [152]:
# Initial position model

# Distribution to sample uniformly from a Julia Set
using Tasks2D.Distributions: uniform_from_set


struct State
    world::GridWorlds.GridWorld
    pos::Vector{Float64}
    t::Int
    hit_wall::Bool
end

# Initialize internal map by sampling each grid cell from iid Bernoulli
# and initialize position by uniformly sampling from empty location
@gen (static) function uniform_agent_pos(t_to_params)
    params = t_to_params(0)
    width, height, wall_prob = params.world.width, params.world.height, params.world.wall_prob

    # draw each grid iid
    world ~ bernoulli_map(width, height, wall_prob)
    cell ~ uniform_from_set(GridWorlds.empty_cells(world))

    # Cell (i, j) corresponds to the region from i-1 to i and j-1 to j
    x ~ uniform(cell[1] - 1, cell[1])
    y ~ uniform(cell[2] - 1, cell[2])

    return State(world, [x, y], 0, false)
end

var"##StaticGenFunction_uniform_agent_pos#2274"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

Next, we define a motion model describing a distribution over the agent's next position,
given its previous positon and the action it takes.

In [153]:
function det_next_pos(pos, a, Δ)
    (x, y) = pos
    a == :up ? [x, y + Δ] :
    a == :down ? [x, y - Δ] :
    a == :left ? [x - Δ, y] :
    a == :right ? [x + Δ, y] :
    a == :stay ? [x, y] :
    error("Unrecognized action: $a")
end

function handle_wall_intersection(prev, new, gridworld)
    walls = GridWorlds.nonempty_segments(gridworld)
    move = L.Segment(prev, new)

    min_collision_dist = Inf
    vec_to_min_dist_collision = nothing
    for i in 1:(size(walls)[1])
        wall = walls[i, :]
        # leverage ray caster to check for collisions in the
        # movement direction
        do_intersect, dist = L.Geometry.cast(move, L.Segment(wall))

        if do_intersect && dist ≤ L.Geometry.norm(move)
            if dist < min_collision_dist
                min_collision_dist = dist
                vec_to_min_dist_collision = L.Geometry.diff(move)
            end
        end
    end

    if !isnothing(vec_to_min_dist_collision)
        dist = min_collision_dist
        if dist < 0.05
            return (prev, true)
        else
            # obtain point of collision by moving forward along the
            # movement vector by a `dist` amount
            normalized_vec = (vec_to_min_dist_collision / L.Geometry.norm(vec_to_min_dist_collision))
            collision_pt = prev + (dist - 0.04) * normalized_vec
            return (collision_pt, true)
        end
    end

    return (new, false)
end

handle_wall_intersection (generic function with 1 method)

In [154]:
# Motion model

@gen (static) function motion_model(state::State, action, t_to_params)
    current_t = state.t + 1
    (a, wall_clock_time) = action
    params = t_to_params(current_t)

    next_pos_det = det_next_pos(state.pos, a, params.step.Δ)
    noisy_next_pos ~ broadcasted_normal(next_pos_det, params.step.σ)
    (next_pos, hit_wall) = handle_wall_intersection(state.pos, noisy_next_pos, state.world)

    return State(state.world, next_pos, current_t, state.hit_wall || hit_wall)
end


var"##StaticGenFunction_motion_model#2305"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

Finally, we define an observation model.  We model the agent making noisy LIDAR distance measurements
along `n_rays` evenly spaced angles.  We model sensor error as Gaussian noise in the measured distances.

In [160]:
@gen function observe_noisy_distances(state::State, t_to_params)
    params = t_to_params(state.t)

    p = reshape([state.pos..., params.obs.orientation], (1, 3))
    _as = L.create_angles(params.obs.fov, params.obs.n_rays)
    wall_segs = GridWorlds.wall_segments(state.world)
    strange_segs = GridWorlds.strange_segments(state.world)

    # handle observations from walls
    w, s_noise, outlier, outlier_vol, zmax = params.obs.wall_sensor_args
    dists_walls = L.cast(p, wall_segs; num_a=params.obs.n_rays, zmax)
    dists_walls = reshape(dists_walls, (:,))


    # handle observations from the strange objects
    w, s_noise, outlier, outlier_vol, zmax = params.obs.strange_sensor_args
    dists_strange = L.cast(p, strange_segs; num_a=params.obs.n_rays, zmax)
    dists_strange = reshape(dists_strange, (:,))

    # check for occlusion
    is_wall_measurement = [w < s for (w, s) in zip(dists_walls, dists_strange)]
    dists_walls = dists_walls[is_wall_measurement]
    dists_strange = dists_strange[.!is_wall_measurement]

    dists_strange_mins = [params.obs.strange_sensor_args.dist_to_zmin(d) for d in dists_strange]
    dists_strange_maxs = [params.obs.strange_sensor_args.dist_to_zmax(d) for d in dists_strange]

    # sample observations
    obs ~ mixture_measurement(
        is_wall_measurement,
        dists_walls,
        params.obs.wall_sensor_args.σ,
        dists_strange_mins,
        dists_strange_maxs,
    )

    return obs
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[State, Any], false, Union{Nothing, Some{Any}}[nothing, nothing], var"##observe_noisy_distances#2405", Bool[0, 0], false)

We can define a POMDP where the state at each timestep is the agent's (x, y) position, and each observation is a vector of noisy distances measured by the agent's "LIDAR" sensor.

For now, we won't have a meaningful utility function.

In [161]:
# POMDP of this environment
pomdp = GenPOMDPs.GenPOMDP(
    uniform_agent_pos,       # INIT   : params                      ⇝ state
    motion_model,            # STEP   : prev_state, actions, params ⇝ state
    observe_noisy_distances, # OBS    : state, params               ⇝ observation
    (state, action) -> 0.0   # UTILITY: state, action, params       → utility
)

GenPOMDPs.GenPOMDP(var"##StaticGenFunction_uniform_agent_pos#2274"(Dict{Symbol, Any}(), Dict{Symbol, Any}()), var"##StaticGenFunction_motion_model#2305"(Dict{Symbol, Any}(), Dict{Symbol, Any}()), DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[State, Any], false, Union{Nothing, Some{Any}}[nothing, nothing], var"##observe_noisy_distances#2405", Bool[0, 0], false), var"#461#462"())

We can use the GenPOMDP library to construct a probability distribution over trajectories for this POMDP, given a fixed sequence of actions the agent takes.

This probability distribution is represented as a Gen generative function.

In [162]:
# Generative function over trajectories of the POMDP,
# given a fixed action sequence.
trajectory_model = GenPOMDPs.ControlledTrajectoryModel(pomdp)

GenPOMDPs.var"##StaticGenFunction__ControlledTrajectoryModel#2477"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

## Interacting with the enviornment

Let's load a gridworld map obtained by tracing a PNG file from the HouseExpo dataset.
We also need to set some parameters controlling the motion and observation model.

In [163]:
_get_params(map) = (;
    # map = GridWorlds.load_custom_map(5),
    # map=map,
    world=(; width=size(map)[1], height=size(map)[2], wall_prob=(1 - length(GridWorlds.empty_cells(map)) / prod(size(map)))),
    step=(; Δ=0.5, σ=0.005),
    obs=(; fov=2π, n_rays=90, orientation=π / 2,
        # I think currently only σ is used
        wall_sensor_args=(;
            w=5, s_noise=0.02,
            outlier=0.0001,
            outlier_vol=100.0,
            zmax=100.0, σ=0.005
        ), strange_sensor_args=(;
            w=5, s_noise=0.02,
            outlier=0.0001,
            outlier_vol=100.0,
            zmax=100.0, σ=5.5,
            dist_to_zmin=x -> 1.0,
            dist_to_zmax=x -> 2.0 + x * 2.5,
        )
    )
)

constant_t_to_params(params) = (t) -> params

MAP_INTRO = GridWorlds.load_houseexpo_gridworld(24, 1)
MAP_EXPLAIN_STRANGE = GridWorlds.load_custom_map(3)
MAP_W_STRANGE = GridWorlds.load_custom_map(4)
MAP_WOUT_STRANGE = GridWorlds.load_custom_map(5)
MAP2_W_STRANGE = GridWorlds.load_custom_map(6)
MAP2_WOUT_STRANGE = GridWorlds.load_custom_map(7)

T_TO_PARAMS_INTRO = constant_t_to_params(_get_params(MAP_INTRO))
T_TO_PARAMS_EXPLAIN_STRANGE = constant_t_to_params(_get_params(MAP_EXPLAIN_STRANGE))
T_TO_PARAMS_W_STRANGE = constant_t_to_params(_get_params(MAP_W_STRANGE))
T_TO_PARAMS_WOUT_STRANGE = constant_t_to_params(_get_params(MAP_WOUT_STRANGE))
T_TO_PARAMS_LEARN_MAP = T_TO_PARAMS_WOUT_STRANGE

#467 (generic function with 1 method)

Then, we can use functionality from the GenPOMDP library to construct an interactive world state, which
we can take actions in by calling a callback function.

Specifically, `GenPOMDPs.interactive_world_trace` will give us a `take_action` callback,
and a `world_trace` object.  The `world_trace` object is an `Observable` from the Julia
`Observables` library.  An `Observable` is a pointer to an object, which also maintains a list of
"subscriber" objects.  Each time the value at the pointer changes, the Observable notifies its
subscribers of the change.

In this case, the `world_trace` object is automatically updated every time the `take_action` callback
is called with an action.  The `world_trace` is an observable over a `Trace` from Gen, for the
`trajectory_model` generative function defined above.  This trace represents a trajectory in the
POMDP, over `T` timesteps.  Each time we call `take_action(a)`, the world trace is automatically
extended by another timestep, by taking action `a`, and simulating a new random world state
and observation from the POMDP.

In [164]:
world_trace, take_action = GenPOMDPs.interactive_world_trace(
    trajectory_model, T_TO_PARAMS_INTRO,
    choicemap(
        (GenPOMDPs.state_addr(0, :cell), (3, 3)),
        (GenPOMDPs.state_addr(0, :world), MAP_INTRO),
    )
);

The GridWorlds library provides visualizations which map keyboard events onto the `take_action` function above.

Use WASD to move the agent around.  Press E to have a timestep pass where the agent takes a "stay" action.
Use G to backtrack in time to view past world states.  Use T to increment time back to the current moment.  (Actions can only be taken when in the farthest simulated world state.)

In [165]:
# Map the world_trace observable to an observable
# over the sequence of ground truth positions, and observation
# point clouds.
function get_posobs_seq(groundtruth_trace)
    return map(
        trace -> (
            [state.pos for state in GenPOMDPs.state_sequence(trace)],
            GenPOMDPs.observation_retval_sequence(trace)
        ),
        groundtruth_trace
    )
end

function get_did_hitwall_observable(trace)
    return map(trace -> GenPOMDPs.state_sequence(trace)[end].hit_wall, trace)
end
function close_window(f)
    glfw_window = GLMakie.to_native(display(f))
    GLMakie.GLFW.SetWindowShouldClose(glfw_window, true)
end
function get_action_times_observable(trace)
    return map(trace -> [t for (a, t) in GenPOMDPs.action_sequence(trace)], trace)
end
function get_timing_args(trace; speedup_factor=1, max_delay=5) # 5 seconds max delay
    return (get_action_times_observable(trace), speedup_factor, max_delay)
end

get_timing_args (generic function with 1 method)

In [166]:

(f, t, actions) = GridWorlds.Viz.interactive_gui(
    t -> MAP_INTRO, get_posobs_seq(world_trace), a -> take_action((a, Dates.now())),
    show_lines_to_walls=false,
    framerate=8,
    close_on_hitwall=true,
    did_hitwall_observable=get_did_hitwall_observable(world_trace),
    close_window=close_window,

    # For playback
    timing_args=get_timing_args(world_trace; speedup_factor=1, max_delay=5)
)
display(f)

GLMakie.Screen(...)

## Localization via Particle Filtering

Now, we'll set up another interactive session.  In this interactive session, we will route the
taken actions, and the observations generated in the POMDP simulation, to a particle filter.
We'll visualize the particles, representing the agent's location, as green squares.  The alpha
of the color will reflect the particles' weights.

For this first demo, we'll just use a bootstrap particle filter.

In [179]:
# New interactive session.
world_trace2, take_action2 = GenPOMDPs.interactive_world_trace(
    trajectory_model, T_TO_PARAMS_INTRO,
    choicemap(
        (GenPOMDPs.state_addr(0, :cell), (3, 3)),
        (GenPOMDPs.state_addr(0, :world), MAP_INTRO),
    )
);

# Construct an observable of the sequence of observations and actions from the
# world (consumed by inference).
function get_actobs_seq(groundtruth_trace)
    return map(
        trace -> (
            GenPOMDPs.observation_sequence(trace),
            GenPOMDPs.action_sequence(trace)
        ),
        groundtruth_trace
    )
end


# Maintain a 200-particle bootstrap particle filter,
# which updates whenever `world_trace2` updates
bootstrap_pf_states = GenPOMDPs.pf_observable(
    GenPOMDPs.bootstrap_pf(
        pomdp,  # Run inference in the ground truth POMDP
        T_TO_PARAMS_INTRO, # with the ground-truth parameters
        200     # using 200 particles
    ),
    get_actobs_seq(world_trace2)
);

# Convert a vector of particle filter states to a vector of
# (particle filter weights, particle filter positions),
# for each timestep.  (This is the format consumed by the visualization.)
get_positions(pf_state) = map(
    (x -> x[end].pos) ∘ GenPOMDPs.state_sequence,
    GenParticleFilters.get_traces(pf_state)
)
function pf_states_to_posweights(pf_states)
    return map(states -> ( # Map the observable to an observable over a tuple...

            # First element of the tuple: particle weights for each timestep.
            map(GenParticleFilters.get_norm_weights, states),

            # Second element of the tuple: particle positions for each timestep
            map(get_positions, states)), pf_states)
end

# Display an interactive GUI, with the particles
(f2, t2, actions2) = GridWorlds.Viz.interactive_gui(
    t -> MAP_INTRO, get_posobs_seq(world_trace2), a -> take_action2((a, Dates.now())),
    show_lines_to_walls=false,
    framerate=8,
    close_on_hitwall=true,
    did_hitwall_observable=get_did_hitwall_observable(world_trace),
    close_window=close_window,

    # For playback
    timing_args=get_timing_args(world_trace; speedup_factor=1, max_delay=5)
)
GridWorlds.Viz.display_pf_localization!(f2, t2, pf_states_to_posweights(bootstrap_pf_states))
display(f2)


GLMakie.Screen(...)

## Inferring the "Strange Object"

In [32]:
import Makie
display(Makie.hbox(f, f2))

LoadError: UndefVarError: `hbox` not defined

## Legacy stuff down below ~~~~~~~~

## SLAM via Particle Filtering

Assuming that the robot doesn't know the map ahead of time, will it be able to reconstruct 
the map and localize itself simutaneously? To test this out, let's set up another generative
model to reflects the robot's internal beliefs and defines how it transists over time.

In [17]:


@gen (static) function uniform_agent_pos_and_world(params)
    w, h, wall_prob = params.world.width, params.world.height, params.world.wall_prob

    # draw each grid iid
    world ~ generate_iid_bernoulli_map(w, h, wall_prob)
    pos ~ uniform_from_set(GridWorlds.empty_cells(world))
    return State(world, pos)
end

var"##StaticGenFunction_uniform_agent_pos_and_world#565"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

In [23]:
state = uniform_agent_pos_and_world((world=(height=10, width=16, wall_prob=0.5),))

f = GridWorlds.Viz.visualize_grid(GridWorlds.Viz.place_agent(state.world, state.pos))
display(f)

GLMakie.Screen(...)

In [24]:
# For step/transition, let's define some utility functions to randomly
# flip the grids in the previous map based on a flip_prob
@gen (static) function flip_wall(flip_prob::Real, cell::GridWorlds.GridCell)
    should_flip ~ bernoulli(flip_prob)
    return should_flip ? (cell == GridWorlds.empty ? GridWorlds.wall : GridWorlds.empty) : cell
end

@gen (static) function update_iid_bernoulli_map(world::GridWorlds.GridWorld, flip_prob::Real)
    width, height = world.size
    raw_map ~ Map(Map(flip_wall))(fill(fill(flip_prob, height), width), world.cells)
    return GridWorlds.FGridWorld(raw_map, nothing, world.size)
end

var"##StaticGenFunction_update_iid_bernoulli_map#702"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

In [25]:
@gen (static) function moving_and_mapping(state::State, action, params)
    σ, flip_prob = params.step.σ, params.world.flip_prob
    world, pos = state.world, state.pos

    # Step 1: fix pos, update map
    # TODO: see if it's necessary to pass in pos as well so that the currently
    # occupied cell won't be turned into wall (it's possible that pf can filter
    # this out)
    world ~ update_iid_bernoulli_map(world, flip_prob)

    # Step 2: update agent pos
    new_agent_position_deterministic = GridWorlds.newpos(world, pos, action)
    (possible_positions, logprobs) = _get_motion_logprobs(
        world, new_agent_position_deterministic, σ
    )

    x = (@assert valid_logprobs(logprobs) "logprobs: $logprobs")

    pos ~ list_categorical(
        possible_positions,
        _normalized_probs(logprobs)
    )

    return State(world, pos)
end

var"##StaticGenFunction_moving_and_mapping#744"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

In [26]:
# let's visualize how an updated state may look like
updated_state = moving_and_mapping(state, :right, (step=(σ=0.3,), world=(flip_prob=0.1,)))
println(state.pos, updated_state.pos)

f = GridWorlds.Viz.visualize_grid(GridWorlds.Viz.place_agent(updated_state.world, updated_state.pos))
display(f)

(6, 2)(7, 2)


GLMakie.Screen(...)

In [27]:
# the observation model is pretty much the same as before
@gen (static) function observe_noisier_distances(state::State, params)
    n_rays, σ = params.obs.n_rays, params.obs.σ

    world_with_agent = GridWorlds.place_agent(state.world, state.pos)
    dists = GridWorlds.ray_trace_distances(cast, world_with_agent, n_rays)
    obs ~ broadcasted_normal(dists, σ)
    return obs
end

var"##StaticGenFunction_observe_noisier_distances#772"(Dict{Symbol, Any}(), Dict{Symbol, Any}())

In [28]:
# Now, let's set up a new POMPDPS for this model
slam_pomdp = GenPOMDPs.GenPOMDP(
    uniform_agent_pos_and_world,   # INIT   : params                      ⇝ state
    moving_and_mapping,            # STEP   : prev_state, actions, params ⇝ state
    observe_noisier_distances,     # OBS    : state, params               ⇝ observation
    (state, action) -> 0.0          # UTILITY: state, action, params       → utility
)

GenPOMDPs.GenPOMDP(var"##StaticGenFunction_uniform_agent_pos_and_world#565"(Dict{Symbol, Any}(), Dict{Symbol, Any}()), var"##StaticGenFunction_moving_and_mapping#744"(Dict{Symbol, Any}(), Dict{Symbol, Any}()), var"##StaticGenFunction_observe_noisier_distances#772"(Dict{Symbol, Any}(), Dict{Symbol, Any}()), var"#250#251"())

In [29]:
width, height = size(PARAMS.map)
SLAM_PARAMS = (
    # load houseexpo env #7,
    # discretized with 24 gridsquares along the x axis
    world=(width, height, wall_prob=0.4, flip_prob=0.1), obs=PARAMS.obs,
    step=PARAMS.step
);

Now, we'll set up another interactive session.  In this interactive session, we will route the
taken actions, and the observations generated in the POMDP simulation, to a particle filter.
We'll visualize the particles, representing the agent's location, as green squares.  The alpha
of the color will reflect the particles' weights.

For this first demo, we'll just use a bootstrap particle filter.

In [92]:
# New interactive session
world_trace2, take_action2 = GenPOMDPs.interactive_world_trace(trajectory_model, PARAMS);

# Construct an observable of the sequence of observations and actions from the
# world (consumed by inference).
function get_actobs_seq(groundtruth_trace)
    return map(
        trace -> (
            GenPOMDPs.observation_sequence(trace),
            GenPOMDPs.action_sequence(trace)
        ),
        groundtruth_trace
    )
end

# Maintain a 200-particle bootstrap particle filter,
# which updates whenever `world_trace2` updates
bootstrap_pf_states = GenPOMDPs.pf_observable(
    GenPOMDPs.bootstrap_pf(
        slam_pomdp,  # Run inference with SLAM
        SLAM_PARAMS, # with the ground-truth parameters
        200     # using 200 particles
    ),
    get_actobs_seq(world_trace2)
);

# Convert a vector of particle filter states to a vector of
# (particle filter weights, particle filter positions),
# for each timestep.  (This is the format consumed by the visualization.)
get_states(pf_state) = map(
    (x -> x[end]) ∘ GenPOMDPs.state_sequence,
    GenParticleFilters.get_traces(pf_state)
)
function pf_states_to_stateweights(pf_states; field=nothing)
    get_fields(states) = field == nothing ? states : map(state -> getfield(state, field), states)
    return map(states -> ( # Map the observable to an observable over a tuple...

            # First element of the tuple: particle weights for each timestep.
            map(GenParticleFilters.get_norm_weights, states),

            # Second element of the tuple: particle states for each timestep
            map(get_fields ∘ get_states, states)), pf_states)
end

# Display an interactive GUI, with the particles|
(f2, t2) = GridWorlds.Viz.interactive_gui(t -> PARAMS.map, get_posobs_seq(world_trace2), take_action2)
GridWorlds.Viz.display_pf_localization!(f2, t2, pf_states_to_stateweights(bootstrap_pf_states; field=:pos))
display(f2)

LoadError: UndefVarError: `PARAMS` not defined

In [ ]:
GridWorlds.Viz.display_pf_state(t2, pf_states_to_stateweights(bootstrap_pf_states))